In [1]:
import datasets
import collections
import random

/home/liuyewei/miniconda3/envs/metalora/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data_all_list = []

In [3]:
def save(data, save_path):
    import json, os
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with open(save_path, 'w') as f:
        for item in data:
            id = item['id']
            context = item['context']
            conversations = []
            for qa in item['qa_pairs']:
                question = qa['question']
                answer = qa['answer']
                conversations.append({"role": "user", "content": question})
                conversations.append({"role": "assistant", "content": answer})
            f.write(json.dumps({"id": id, "context": context, "conversations": conversations}) + "\n")

In [4]:
# COQA ('stanfordnlp/coqa')
output_list = []
data = datasets.load_dataset('stanfordnlp/coqa')
data = list(data['train'])
# iterate all items in the dataset

random.seed(0)

item_list = []
for i, item in enumerate(data):
    context = item['story']
    questions = item['questions']
    answers = item['answers']["input_text"]
    item_list.append({
        'context': context,
        'qa_pairs': [{'question': q, 'answer': a} for q, a in zip(questions, answers)]
    })

# randomly merge five of each to one item
num_context = 8
num_merge = 1
for i in range(0, len(item_list), num_merge):
    if i + num_merge > len(item_list):
        break
    context_list = [item_list[j]['context'] for j in range(i, i + num_merge)]
    context_list += [item['context'] for item in random.sample(item_list, random.randint(0, num_context))]
    random.shuffle(context_list)
    context = "\n\n".join(context_list)
    if len(context.split()) > 4096:
        print(f"skip context len: {len(context.split())}")
        continue
    qa_pairs_list = [item_list[j]['qa_pairs'] for j in range(i, i + num_merge)]
    random.shuffle(qa_pairs_list)
    qa_pairs = sum(qa_pairs_list, [])
    while sum([len(qa["question"].split()) for qa in qa_pairs]) + sum([len(qa["answer"].split()) for qa in qa_pairs]) > 512:
        qa_pairs = qa_pairs[:-1]
        # print(f"q eln: {[len(qa['question'].split()) for qa in qa_pairs]}, a len: {[len(qa['answer'].split()) for qa in qa_pairs]}")
        # continue
    output_list.append({
        "id": f"coqa.{len(output_list):04d}",
        'context': context,
        'qa_pairs': qa_pairs
    })

print(f"COQA: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
print(collections.Counter([len(item["qa_pairs"]) for item in output_list]))
save(output_list, 'v4/coqa.jsonl')
# data_all_list.extend(output_list)
# # output_list

COQA: 7199, context len: 1357.6557855257674
Counter({20: 2452, 10: 1070, 11: 463, 12: 410, 13: 337, 15: 327, 19: 304, 18: 296, 14: 283, 17: 233, 16: 209, 21: 148, 1: 119, 2: 91, 6: 71, 3: 65, 5: 54, 9: 54, 7: 51, 4: 50, 8: 48, 22: 33, 23: 16, 27: 3, 24: 3, 36: 2, 28: 2, 25: 2, 26: 1, 30: 1, 29: 1})


In [5]:
# DROP ('ucinlp/drop')
output_list = []
data = datasets.load_dataset('ucinlp/drop')
data = list(data['train'])
random.seed(0)

context_to_item = {}
for item in data:
    context = item['passage']
    if context not in context_to_item:
        context_to_item[context] = []
    context_to_item[context].append(item)


item_list = []
for i, context in enumerate(context_to_item):
    qa_pairs = []
    for item in context_to_item[context]:
        question = item['question']
        answer = item['answers_spans']['spans'][0]
        qa_pairs.append({'question': question, 'answer': answer})
    item_list.append({
        'context': context,
        'qa_pairs': qa_pairs,
    })


num_context = 8
num_merge = 1
for i in range(0, len(item_list), num_merge):
    if i + num_merge > len(item_list):
        break
    context_list = [item_list[j]['context'] for j in range(i, i + num_merge)]
    context_list += [item['context'] for item in random.sample(item_list, random.randint(0, num_context))]
    random.shuffle(context_list)
    context = "\n\n".join(context_list)
    if len(context.split()) > 4096:
        print(f"skip context len: {len(context.split())}")
        continue
    qa_pairs_list = [item_list[j]['qa_pairs'] for j in range(i, i + num_merge)]
    random.shuffle(qa_pairs_list)
    qa_pairs = sum(qa_pairs_list, [])
    while sum([len(qa["question"].split()) for qa in qa_pairs]) + sum([len(qa["answer"].split()) for qa in qa_pairs]) > 512:
        qa_pairs = qa_pairs[:-1]
        # print(f"q eln: {[len(qa['question'].split()) for qa in qa_pairs]}, a len: {[len(qa['answer'].split()) for qa in qa_pairs]}")
        # continue
    output_list.append({
        "id": f"drop.{len(output_list):04d}",
        'context': context,
        'qa_pairs': qa_pairs
    })

print(f"DROP: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
print(collections.Counter([len(item["qa_pairs"]) for item in output_list]))
save(output_list, 'v4/drop.jsonl')
# data_all_list.extend(output_list)
# output_list

DROP: 5541, context len: 1067.5722793719544
Counter({7: 306, 5: 292, 8: 276, 9: 270, 3: 266, 4: 265, 6: 264, 11: 259, 13: 249, 12: 230, 14: 223, 15: 218, 2: 205, 17: 189, 16: 181, 10: 172, 1: 164, 18: 153, 19: 133, 20: 122, 21: 110, 22: 107, 23: 105, 24: 86, 25: 80, 26: 69, 27: 61, 28: 51, 31: 43, 29: 41, 34: 38, 32: 36, 30: 30, 35: 29, 37: 26, 38: 25, 39: 23, 41: 19, 33: 19, 42: 18, 40: 17, 36: 17, 43: 15, 44: 13, 46: 8, 47: 7, 45: 6, 51: 2, 54: 1, 49: 1, 48: 1})


In [6]:
# narrativeqa ('deepmind/narrativeqa')
output_list = []
data = datasets.load_dataset('deepmind/narrativeqa')
data = list(data['train'])
context_to_item = {}
for item in data:
    context = item['document']['summary']['text']
    if context not in context_to_item:
        context_to_item[context] = []
    context_to_item[context].append(item)
for i, context in enumerate(context_to_item):
    if len(context.split()) > 2048:
        print(f"skip context len: {len(context.split())}")
        continue
    qa_pairs = []
    for item in context_to_item[context]:
        question = item['question']['text']
        answer = item['answers'][0]['text']
        qa_pairs.append({'question': question, 'answer': answer})
    while sum([len(qa["question"].split()) for qa in qa_pairs]) + sum([len(qa["answer"].split()) for qa in qa_pairs]) > 512:
        qa_pairs = qa_pairs[:-1]
        # print(f"q eln: {[len(qa['question'].split()) for qa in qa_pairs]}, a len: {[len(qa['answer'].split()) for qa in qa_pairs]}")
        # continue
    output_list.append({
        "id": f"narrativeqa.{i:04d}",
        'context': context,
        'qa_pairs': qa_pairs
    })
print(f"narrativeqa: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
print(collections.Counter([len(item["qa_pairs"]) for item in output_list]))
save(output_list, 'v4/narrativeqa.jsonl')
# output_list

Generating validation split: 100%|██████████| 3461/3461 [00:02<00:00, 1408.18 examples/s]


narrativeqa: 1102, context len: 577.0644283121597
Counter({30: 537, 29: 325, 28: 131, 27: 47, 26: 23, 40: 11, 38: 4, 37: 3, 25: 3, 31: 3, 32: 3, 20: 2, 39: 2, 36: 1, 33: 1, 24: 1, 45: 1, 42: 1, 48: 1, 35: 1, 23: 1})


In [7]:
# PubMed ('qiaojin/PubMedQA')
output_list = []
data = datasets.load_dataset('qiaojin/PubMedQA', 'pqa_labeled')
data = list(data['train'])
context_to_item = {}
for item in data:
    context = "\n".join(item['context']['contexts'])
    if context not in context_to_item:
        context_to_item[context] = []
    context_to_item[context].append(item)
for i, context in enumerate(context_to_item):
    if len(context.split()) > 2048:
        print(f"skip context len: {len(context.split())}")
        continue
    qa_pairs = []
    assert len(context_to_item[context]) == 1
    for item in context_to_item[context]:
        question = item['question']
        answer = f'{item["final_decision"].capitalize()}. {item["long_answer"]}'
        qa_pairs.append({'question': question, 'answer': answer})
    if sum([len(qa["question"].split()) for qa in qa_pairs]) + sum([len(qa["answer"].split()) for qa in qa_pairs]) > 512:
        print(f"q eln: {[len(qa['question'].split()) for qa in qa_pairs]}, a len: {[len(qa['answer'].split()) for qa in qa_pairs]}")
        continue
    output_list.append({
        "id": f"pubmedqa.{i:04d}",
        'context': context,
        'qa_pairs': qa_pairs
    })
print(f"pubmedqa: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
print(collections.Counter([len(item["qa_pairs"]) for item in output_list]))
save(output_list, 'v4/pubmedqa.jsonl')
# data_all_list.extend(output_list)
# output_list

Generating train split: 100%|██████████| 1000/1000 [00:00<00:00, 45665.21 examples/s]


pubmedqa: 1000, context len: 200.207
Counter({1: 1000})


In [8]:
# quail ('textmachinelab/quail')
output_list = []
data = datasets.load_dataset('textmachinelab/quail')
data = list(data['train'])
context_to_item = {}
for item in data:
    context = item['context']
    if context not in context_to_item:
        context_to_item[context] = []
    context_to_item[context].append(item)
for i, context in enumerate(context_to_item):
    if len(context.split()) > 2048:
        print(f"skip context len: {len(context.split())}")
        continue
    qa_pairs = []
    for item in context_to_item[context]:
        question = item['question']
        answer = item['answers'][item['correct_answer_id']]
        if answer == "not enough information":
            continue
        qa_pairs.append({'question': question, 'answer': answer})
    if len(qa_pairs) == 0:
        continue
    if sum([len(qa["question"].split()) for qa in qa_pairs]) + sum([len(qa["answer"].split()) for qa in qa_pairs]) > 512:
        print(f"q eln: {[len(qa['question'].split()) for qa in qa_pairs]}, a len: {[len(qa['answer'].split()) for qa in qa_pairs]}")
        continue
    output_list.append({
        "id": f"quail.{i:04d}",
        'context': context,
        'qa_pairs': qa_pairs
    })
print(f"quail: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
print(collections.Counter([len(item["qa_pairs"]) for item in output_list]))
save(output_list, 'v4/quail.jsonl')
# output_list

Generating challenge split: 100%|██████████| 556/556 [00:00<00:00, 65293.79 examples/s]


quail: 560, context len: 332.6714285714286
Counter({16: 522, 17: 14, 18: 7, 23: 5, 19: 4, 24: 3, 26: 2, 22: 2, 20: 1})


In [9]:
# # quail ('rajpurkar/squad_v2')
# output_list = []
# data = datasets.load_dataset('rajpurkar/squad_v2')
# data = list(data['train'])
# context_to_item = {}
# for item in data:
#     context = item['context']
#     if context not in context_to_item:
#         context_to_item[context] = []
#     context_to_item[context].append(item)
# for i, context in enumerate(context_to_item):
#     if len(context.split()) > 1024:
#         print(f"skip context len: {len(context.split())}")
#         continue
#     qa_pairs = []
#     for item in context_to_item[context]:
#         if len(item['answers']["text"]) == 0:
#             continue
#         question = item['question']
#         answer = item['answers']["text"][0]
#         qa_pairs.append({'question': question, 'answer': answer})
#     if len(qa_pairs) == 0:
#         continue
#     output_list.append({
#         "id": f"squad_v2.{i:04d}",
#         'context': context,
#         'qa_pairs': qa_pairs
#     })
# print(f"squad_v2: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
# print(collections.Counter([len(item["qa_pairs"]) for item in output_list]))
# data_all_list.extend(output_list)
# # output_list

In [10]:
# ms marco ('microsoft/ms_marco')
output_list = []
data = datasets.load_dataset('microsoft/ms_marco', 'v1.1')
data = list(data['train'])
random.seed(0)

context_to_item = {}
for item in data:
    passages = item['passages']
    if sum(passages['is_selected']) == 0:
        continue
    is_selected_idx = passages['is_selected'].index(1)
    assert isinstance(is_selected_idx, int)
    context = passages['passage_text'][is_selected_idx]
    if context not in context_to_item:
        context_to_item[context] = []
    context_to_item[context].append(item)
item_list = []
for i, context in enumerate(context_to_item):
    qa_pairs = []
    for item in context_to_item[context]:
        if len(item['answers']) == 0:
            continue
        question = item['query']
        answer = item['answers'][0]
        qa_pairs.append({'question': question, 'answer': answer})
    if len(qa_pairs) == 0:
        continue
    item_list.append({
        'context': context,
        'qa_pairs': qa_pairs,
    })
# randomly merge five of each to one item
num_context = 32
num_merge = 4
for i in range(0, len(item_list), num_merge):
    if i + num_merge > len(item_list):
        break
    context_list = [item_list[j]['context'] for j in range(i, i + num_merge)]
    context_list += [item['context'] for item in random.sample(item_list, random.randint(0, num_context))]
    random.shuffle(context_list)
    context = "\n\n".join(context_list)
    if len(context.split()) > 4096:
        print(f"skip context len: {len(context.split())}")
        continue
    qa_pairs_list = [item_list[j]['qa_pairs'] for j in range(i, i + num_merge)]
    random.shuffle(qa_pairs_list)
    qa_pairs = sum(qa_pairs_list, [])
    while sum([len(qa["question"].split()) for qa in qa_pairs]) + sum([len(qa["answer"].split()) for qa in qa_pairs]) > 512:
        qa_pairs = qa_pairs[:-1]
        # print(f"q eln: {[len(qa['question'].split()) for qa in qa_pairs]}, a len: {[len(qa['answer'].split()) for qa in qa_pairs]}")
        # continue
    output_list.append({
        "id": f"msmarco.{len(output_list):04d}",
        'context': context,
        'qa_pairs': qa_pairs
    })

print(f"msmarco: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
print(collections.Counter([len(item["qa_pairs"]) for item in output_list]))
save(output_list, 'v4/msmarco.jsonl')

# data_all_list.extend(output_list)
# output_list

Generating test split: 100%|██████████| 9650/9650 [00:00<00:00, 35106.85 examples/s]


msmarco: 19331, context len: 1441.1601055299777
Counter({4: 17541, 5: 1463, 6: 232, 7: 52, 8: 20, 9: 9, 10: 7, 11: 5, 12: 2})


In [11]:
# PwC
output_list = []
data = datasets.load_dataset('sggetao/PwC')
data = list(data['train'])
context_to_item = {}
for item in data:
    context = item['input']
    if context not in context_to_item:
        context_to_item[context] = []
    context_to_item[context].append(item)
for i, context in enumerate(context_to_item):
    if len(context.split()) > 2048:
        print(f"skip context len: {len(context.split())}")
        continue
    qa_pairs = []
    for item in context_to_item[context]:
        if len(item['answer'].split()) > 128 or len(item['prompt'].split()) > 128:
            continue
        question = item['prompt']
        answer = item['answer']
        qa_pairs.append({'question': question, 'answer': answer})
    # randomly sample 5 qa pairs for each context
    # qa_pairs = qa_pairs[:8]
    while sum([len(qa["question"].split()) for qa in qa_pairs]) + sum([len(qa["answer"].split()) for qa in qa_pairs]) > 512:
        qa_pairs = qa_pairs[:-1]
        # print(f"q eln: {[len(qa['question'].split()) for qa in qa_pairs]}, a len: {[len(qa['answer'].split()) for qa in qa_pairs]}")
        continue
    output_list.append({
        "id": f"pwc.{i:04d}",
        'context': context,
        'qa_pairs': qa_pairs
    })
print(f"pwc: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
print(collections.Counter([len(item["qa_pairs"]) for item in output_list]))
save(output_list, 'v4/pwc.jsonl')
# output_list

Generating test split: 100%|██████████| 18146/18146 [00:00<00:00, 190239.41 examples/s]


pwc: 16382, context len: 354.6154926138445
Counter({10: 3951, 11: 3348, 14: 2853, 13: 2582, 12: 1531, 9: 803, 15: 785, 8: 341, 7: 154, 6: 32, 5: 2})


In [13]:
import json, torch, transformers, random
from tqdm import tqdm
tokenizer = transformers.AutoTokenizer.from_pretrained('../models/Qwen3-8B/')

output_list = []
random.seed(0)

with open("../../data/metaicl/hr_to_lr.json", "r") as f:
    config = json.load(f)

for task in tqdm(config["train"]):
    dp = torch.load(f'../../data/metaicl/llama/{task}/{task}_16384_100_train.jsonl')
    k_shot = 64
    k_test = 4
    max_num = 200
    n = len(dp['input'])


    input_text = [tokenizer.decode(dp['input'][i]) for i in range(n)]
    output_text = [tokenizer.decode(dp['output'][i]) for i in range(n)]
    avg_input_len = sum([len(x.split()) for x in input_text]) / n
    k_shot = min(k_shot, 2048 // int(avg_input_len))
    k_test = min(k_test, 512 // int(avg_input_len))
    
    if k_shot == 0 or k_test == 0:
        print(f"skip {task} due to avg_input_len: {avg_input_len}")
        continue

    # reduce the number of examples
    n = min(n, max_num * (k_shot + k_test))
    index = random.sample(range(len(input_text)), n)
    input_text = [input_text[i] for i in index]
    output_text = [output_text[i] for i in index]

    # first k are demonstrations, the rest are test case
    for i in range(0, n, k_shot + k_test):
        if i + k_shot + k_test > n:
            break
        context = "\n\n".join([f"Input: {input_text[j]}\nOutput: {output_text[j]}" for j in range(i, i + k_shot)])
        # test_input_text = f"Input: {input_text[i + k]}\nOutput: "
        # test_output_text = f"{output_text[i + k]}"
        output_list.append({
            "id": f"metaicl.{len(output_list):04d}.{task}.{i // (k_shot + k_test)}",
            'context': context,
            'qa_pairs': [
                {'question': f"Input: {input_text[j]}\nOutput:", 'answer': output_text[j]} for j in range(i + k_shot, i + k_shot + k_test)
            ],
        })
    print(task, n, avg_input_len, k_shot, k_test, len(output_list))
print(f"metaicl: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
print(collections.Counter([len(item["qa_pairs"]) for item in output_list]))
save(output_list, 'v4/metaicl.jsonl')

FileNotFoundError: [Errno 2] No such file or directory: '../../data/metaicl/hr_to_lr.json'

In [14]:
import re

# Long Alpaca
# data_path = "../../data/long-llm/longalpaca/train.json"
# data = datasets.load_dataset('json', data_files=data_path)
data = datasets.load_dataset('Yukang/LongAlpaca-12k')
data = list(data["train"])
output_list = []
for i, item in enumerate(data):
    first_message = item["instruction"]
    second_message = item["output"]
    match_success = False
    try:
        result = re.search(r"^(.*)Now the (.*?) ends\.(.*?)$", first_message, re.DOTALL)
        assert result is not None, f"id: {i}"
        instruction = result.group(3).strip()
        context = first_message[:first_message.index(result.group(3))].strip()
        response = second_message
        match_success = True
    except:
        pass

    if not match_success:
        continue

    # if the lenth of the context exeed 8192, skip
    if len(context.split()) > 6144:
        continue    

    output_list.append({
        "id": f"longalpaca.{i:05d}",
        "context": context,
        "conversations": [
            {"role": "user", "content": instruction},
            {"role": "assistant", "content": response}
        ]
    })
# data_all_list.extend(output_list)
print(f"LONGALPACA: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
print(collections.Counter([len(item["conversations"]) for item in output_list]))
# save output_list to v2/longalpaca.jsonl
with open("v4/longalpaca.jsonl", "w") as f:
    for item in output_list:
        f.write(json.dumps(item) + "\n")

Repo card metadata block was not found. Setting CardData to empty.
Generating train split: 100%|██████████| 12000/12000 [00:06<00:00, 1931.31 examples/s]


LONGALPACA: 4746, context len: 4964.772439949431
Counter({2: 4746})


In [15]:
# booksum
output_list = []
data = datasets.load_dataset('kmfoda/booksum')
data = list(data['train'])
# iterate all items in the dataset
for i, item in enumerate(data):
    assert item is not None
    book_id = item["book_id"].split(".")[0]
    summary_id = item["summary_id"]
    context = f"{book_id}, {summary_id}:\n\n{item['chapter']}"
    question = f"Summarize {book_id}, {summary_id}"
    answer = item['summary_text']
    if len(context.split()) > 4096:
        # print(f"skip context len: {len(context.split())}")
        continue
    if len(question.split()) + len(answer.split()) > 512:
        # print(f"q len: {len(question.split())}, a len: {len(answer.split())}")
        continue
    output_list.append({
        "id": f"booksum.{i:04d}",
        'context': context,
        'qa_pairs': [{'question': question, 'answer': answer}]
    })
print(f"booksum: {len(output_list)}, context len: {sum([len(x['context'].split()) for x in output_list]) / len(output_list)}")
print(collections.Counter([len(item["qa_pairs"]) for item in output_list]))
save(output_list, 'v4/booksum.jsonl')
# data_all_list.extend(output_list)
# # output_list

Generating test split: 100%|██████████| 1431/1431 [00:00<00:00, 5012.12 examples/s]


booksum: 5802, context len: 2058.730782488797
Counter({1: 5802})


In [16]:
# concatenate all "v4/*.jsonl" to "sft-v4.jsonl"
import os
import json
import glob

config = {
    "sft-v4-qa.jsonl": ["coqa", "drop", "narrativeqa", "pubmedqa", "quail", "msmarco"],
    "sft-v4-ift.jsonl": ["coqa", "drop", "narrativeqa", "pubmedqa", "quail", "msmarco", "booksum", "longalpaca"],
    # "sft-v4.jsonl": ["coqa", "drop", "narrativeqa", "pubmedqa", "quail", "msmarco", "booksum", "longalpaca", "metaicl"],
}

for output_path, dataset_names in config.items():
    output_list = []
    for dataset_name in dataset_names:
        with open(f"v4/{dataset_name}.jsonl", "r") as f:
            data = [json.loads(line) for line in f]
        output_list.extend(data)
    with open(output_path, "w") as f:
        for item in output_list:
            f.write(json.dumps(item) + "\n")
    print(f"{output_path}: {len(output_list)}")


sft-v4-qa.jsonl: 34733
sft-v4-ift.jsonl: 45281


In [18]:
import pandas as pd
import json
with open("sft-v4-ift.jsonl", "r") as f:
    output_list = [json.loads(line) for line in f]

# measure "#docs	#instructions	doc len	instruction len	response len"
stat_list = []
for item in output_list:
    dataset = item['id'].split(".")[0]
    n_docs = 1
    n_instructions = sum([x['role'] == 'user' for x in item['conversations']])
    n_responses = sum([x['role'] == 'assistant' for x in item['conversations']])
    doc_len = len(item['context'].split())
    instruction_len_mean = sum([len(x['content'].split()) for x in item['conversations'] if x['role'] == 'user']) / n_instructions
    response_len_mean = sum([len(x['content'].split()) for x in item['conversations'] if x['role'] == 'assistant']) / n_responses
    instruction_len_max = max([len(x['content'].split()) for x in item['conversations'] if x['role'] == 'user'])
    response_len_max = max([len(x['content'].split()) for x in item['conversations'] if x['role'] == 'assistant'])
    instruction_len_min = min([len(x['content'].split()) for x in item['conversations'] if x['role'] == 'user'])
    response_len_min = min([len(x['content'].split()) for x in item['conversations'] if x['role'] == 'assistant'])
    instruction_len_sum = sum([len(x['content'].split()) for x in item['conversations'] if x['role'] == 'user'])
    response_len_sum = sum([len(x['content'].split()) for x in item['conversations'] if x['role'] == 'assistant'])
    stat_list.append({
        "dataset": dataset,
        "n_docs": n_docs,
        "n_instructions": n_instructions,
        "n_responses": n_responses,
        "doc_len": doc_len,
        "instruction_len_mean": instruction_len_mean,
        "response_len_mean": response_len_mean,
        "instruction_len_max": instruction_len_max,
        "response_len_max": response_len_max,
        "instruction_len_min": instruction_len_min,
        "response_len_min": response_len_min,
        "instruction_len_sum": instruction_len_sum,
        "response_len_sum": response_len_sum,
    })

df = pd.DataFrame(stat_list)
df.groupby("dataset").mean()

,n_docs,n_instructions,n_responses,doc_len,instruction_len_mean,response_len_mean,instruction_len_max,response_len_max,instruction_len_min,response_len_min,instruction_len_sum,response_len_sum
dataset,,,,,,,,,,,,
booksum,1.0,1.000000,1.000000,2058.730782,6.656670,228.837125,6.656670,228.837125,6.656670,228.837125,6.656670,228.837125
coqa,1.0,15.091957,15.091957,1357.655786,5.529221,2.710168,9.621475,8.134741,2.221003,1.068482,82.389637,40.053341
drop,1.0,13.469951,13.469951,1067.572279,10.901193,1.398596,16.408591,2.797690,6.790652,1.016604,145.391807,18.761956
longalpaca,1.0,1.000000,1.000000,4964.772440,16.827223,210.755162,16.827223,210.755162,16.827223,210.755162,16.827223,210.755162
msmarco,1.0,4.119032,4.119032,1441.160106,6.024079,14.095591,8.639698,30.951218,3.892039,2.684962,24.830480,57.934820
narrativeqa,1.0,29.438294,29.438294,577.064428,8.580034,4.555363,16.157895,13.985481,3.803993,1.027223,252.284029,133.589837
pubmedqa,1.0,1.000000,1.000000,200.207000,12.913000,40.660000,12.913000,40.660000,12.913000,40.660000,12.913000,40.660000
quail,1.0,16.241071,16.241071,332.671429,8.669452,4.925956,14.942857,11.891071,4.260714,1.194643,140.471429,79.723214


In [ ]:
# df.groupby("dataset").sum()

In [ ]:
# df.groupby("dataset").max()

In [ ]:
# df.groupby("dataset").min()

In [ ]:
# # plot distribution of df["doc_len"] 
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.histplot(df["doc_len"], bins=100)
# plt.show()